In [51]:
import pandas as pd
from prophet import Prophet


In [52]:
df_flights = pd.read_csv('../FlightData/flights_SFO_UA.csv')
df_planes = pd.read_csv('../PlaneData/MainlineFleet.csv')
df_holiday = pd.read_csv('../HolidayData/holidays_2023_2024.csv')
df_future_flights = pd.read_csv('../FutureData/future_flights_data.csv')

# df_weather

In [53]:
df_planes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1834 entries, 0 to 1833
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PLANE_MODEL  1819 non-null   object
 1   TAIL_NUM     1584 non-null   object
dtypes: object(2)
memory usage: 28.8+ KB


In [54]:
import pandas as pd

# Load the CSV files into DataFrames
df_express_fleet = pd.read_csv('../PlaneData/ExpressFleet.csv')
df_mainline_fleet = pd.read_csv('../PlaneData/MainlineFleet.csv')

# Concatenate the DataFrames from the CSV files
df_planes = pd.concat([df_express_fleet, df_mainline_fleet], ignore_index=True)

# Merge the concatenated DataFrame with df_flights
df_flights_plane = df_flights.merge(df_planes[['PLANE_MODEL', 'TAIL_NUM']], how='left', left_on='TAIL_NUM', right_on='TAIL_NUM')

# Print out all unique TAIL_NUM that don't have a PLANE_MODEL
missing_plane_model = df_flights_plane[df_flights_plane['PLANE_MODEL'].isna()]['TAIL_NUM'].unique()
print("Unique TAIL_NUM without a PLANE_MODEL:", missing_plane_model, "count:", len(missing_plane_model))


Unique TAIL_NUM without a PLANE_MODEL: ['N13110' 'N13138' 'N14102' 'N14242' 'N17133' 'N17229' 'N18119' 'N18220'
 'N215UA' 'N222UA' 'N228UA' 'N2331U' 'N2645U' 'N27205' 'N27421' 'N2747U'
 'N29124' 'N29129' 'N34137' 'N37413' 'N37419' 'N37466' 'N37504' 'N38454'
 'N38458' 'N38473' 'N39418' 'N41135' 'N417UA' 'N429UA' 'N437UA' 'N438UA'
 'N445UA' 'N457UA' 'N467UA' 'N468UA' 'N469UA' 'N472UA' 'N476UA' 'N477UA'
 'N48127' 'N487UA' 'N491UA' 'N57111' 'N68807' 'N69819' 'N75426' 'N76503'
 'N76504' 'N76529' 'N76532' 'N77542' 'N779UA' 'N78506' 'N785UA' 'N87512'
 'N14107' 'N14115' 'N14214' 'N14235' 'N19136' 'N19141' 'N21108' 'N2138U'
 'N2142U' 'N220UA' 'N26232' 'N33203' 'N37277' 'N37422' 'N37434' 'N37506'
 'N37510' 'N37513' 'N423UA' 'N441UA' 'N462UA' 'N475UA' 'N496UA' 'N53441'
 'N69806' 'N69816' 'N69818' 'N73259' 'N76514' 'N77430' 'N77535' 'N776UA'
 'N781UA' 'N787UA' 'N793UA' 'N12125' 'N14106' 'N2135U' 'N214UA' 'N27477'
 'N36469' 'N37263' 'N39415' 'N39423' 'N41140' 'N471UA' 'N489UA' 'N76505'
 'N76516' 'N

In [55]:
df_flights_plane['PLANE_MODEL'].unique()

array([nan, '38M', '39M', '787-8', '787-9', '739ER', '753', '763ER',
       '772ER', '319', '752', '77W', '787-10', '764ER', '738'],
      dtype=object)

In [56]:
for origin, dest in df_flights_plane[['ORIGIN_CITY_NAME', 'DEST_CITY_NAME']].drop_duplicates().values:
    # Filter rows for the current combination of ORIGIN_CITY_NAME and DEST_CITY_NAME
    mask = (df_flights_plane['ORIGIN_CITY_NAME'] == origin) & (df_flights_plane['DEST_CITY_NAME'] == dest)
    
    # Find the first non-NaN PLANE_MODEL for the current combination
    plane_model = df_flights_plane.loc[mask, 'PLANE_MODEL'].dropna().unique()
    
    if len(plane_model) > 0:
        # Use the first non-NaN PLANE_MODEL if it exists
        df_flights_plane.loc[mask & df_flights_plane['PLANE_MODEL'].isna(), 'PLANE_MODEL'] = plane_model[0]


In [57]:
# Calculate the percentage of unique TAIL_NUM that have no model
unique_tail_nums = df_flights_plane['TAIL_NUM'].nunique()
unique_tail_nums_no_model = df_flights_plane[df_flights_plane['PLANE_MODEL'].isna()]['TAIL_NUM'].nunique()
percentage_unique_no_model = (unique_tail_nums_no_model / unique_tail_nums) * 100

# Calculate the percentage of all rows that have no model
total_rows = len(df_flights_plane)
rows_no_model = df_flights_plane['PLANE_MODEL'].isna().sum()
percentage_rows_no_model = (rows_no_model / total_rows) * 100

# Print the results
print(f"Percentage of unique TAIL_NUM that have no model: {percentage_unique_no_model:.2f}%")
print(f"Percentage of all rows that have no model: {percentage_rows_no_model:.2f}%")


Percentage of unique TAIL_NUM that have no model: 0.11%
Percentage of all rows that have no model: 0.00%


In [58]:
df_flights_plane

,ORIGIN_CITY_NAME,ORIGIN_AIRPORT_ID,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,TAIL_NUM,FL_DATE,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,DEST_CITY_NAME,DEST_AIRPORT_ID,AIR_TIME,DEP_TIME,DEP_DELAY,DEP_DEL15,PLANE_MODEL
0,"San Francisco, CA",14771,UA,1665,N13110,2024-02-01,2024,2,1,4,"Boston, MA",10721,329.0,23:44:00,19.0,1.0,752
1,"San Francisco, CA",14771,UA,716,N13110,2024-02-01,2024,2,1,4,"Boston, MA",10721,314.0,08:17:00,-8.0,0.0,752
2,"San Francisco, CA",14771,UA,1999,N13138,2024-02-01,2024,2,1,4,"Denver, CO",11292,122.0,11:42:00,36.0,1.0,38M
3,"San Francisco, CA",14771,UA,2374,N14102,2024-02-01,2024,2,1,4,"Boston, MA",10721,314.0,13:30:00,15.0,1.0,752
4,"San Francisco, CA",14771,UA,1041,N14242,2024-02-01,2024,2,1,4,"San Diego, CA",14679,66.0,08:27:00,-8.0,0.0,39M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50235,"San Francisco, CA",14771,UA,246,N850UA,2023-10-31,2023,10,31,2,"Albuquerque, NM",10140,115.0,11:04:00,-6.0,0.0,319
50236,"San Francisco, CA",14771,UA,1581,N872UA,2023-10-31,2023,10,31,2,"Boise, ID",10713,70.0,08:34:00,-11.0,0.0,319
50237,"San Francisco, CA",14771,UA,1640,N881UA,2023-10-31,2023,10,31,2,"Dallas/Fort Worth, TX",11298,184.0,10:58:00,-8.0,0.0,319
50238,"San Francisco, CA",14771,UA,1107,N884UA,2023-10-31,2023,10,31,2,"Austin, TX",10423,185.0,08:37:00,-3.0,0.0,319


In [59]:
df_flights_plane = df_flights_plane.dropna(subset=['PLANE_MODEL'])

In [60]:
flights_plane_csv_path = '/Users/judith.rethmann/Documents/MBS/PredictiveAnalytics/Model/flights_plane.csv'
df_flights_plane.to_csv(flights_plane_csv_path, index=False)
print(f"Combined CSV file saved as '{flights_plane_csv_path}'")


Combined CSV file saved as '/Users/judith.rethmann/Documents/MBS/PredictiveAnalytics/Model/flights_plane.csv'


In [72]:
df_flights_plane.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50239 entries, 0 to 50239
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   FL_DATE         50239 non-null  datetime64[ns]
 1   DAY_OF_WEEK     50239 non-null  int32         
 2   DEST_CITY_NAME  50239 non-null  object        
 3   AIR_TIME        50239 non-null  float64       
 4   DEP_TIME        50239 non-null  object        
 5   DEP_DEL15       50239 non-null  float64       
 6   PLANE_MODEL     50239 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), object(3)
memory usage: 2.9+ MB


In [71]:
df_flights_plane.head()

,FL_DATE,DAY_OF_WEEK,DEST_CITY_NAME,AIR_TIME,DEP_TIME,DEP_DEL15,PLANE_MODEL
0,2024-02-01,3,Boston,329.0,23:44:00,1.0,752
1,2024-02-01,3,Boston,314.0,08:17:00,0.0,752
2,2024-02-01,3,Denver,122.0,11:42:00,1.0,38M
3,2024-02-01,3,Boston,314.0,13:30:00,1.0,752
4,2024-02-01,3,San Diego,66.0,08:27:00,0.0,39M


In [70]:
df_future_flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   FL_DATE         203 non-null    datetime64[ns]
 1   DEST_CITY_NAME  203 non-null    object        
 2   PLANE_MODEL     203 non-null    object        
 3   DEP_TIME        203 non-null    object        
 4   AIR_TIME        203 non-null    int64         
 5   DAY_OF_WEEK     203 non-null    int32         
dtypes: datetime64[ns](1), int32(1), int64(1), object(3)
memory usage: 8.9+ KB


In [64]:
df_future_flights.head()

,clean_date,carrier,flightnumber,airport_name,airport_city_name,airport_IATA,aircraft_name,aircraft_IATA,departure_time,arrival_time,elapsed_time,codeshare_info
0,2024-05-31,UA,512,George Bush Intcntl Houston,Houston,IAH,Boeing 737MAX 9 Passenger,7M9,00:30,06:26:00,236,NZ 9190 /VA 8082
1,2024-05-31,UA,189,Ninoy Aquino International,Manila,MNL,Boeing 777-300ER,77W,00:45,05:55:00,850,NaN
2,2024-05-31,UA,1139,Chicago Ohare International,Chicago,ORD,Boeing 737-900 Passenger,739,01:10,07:30:00,260,AC 3088 /NZ 9308
3,2024-05-31,UA,274,George Bush Intcntl Houston,Houston,IAH,Boeing 737MAX 9 Passenger,7M9,05:00,11:03:00,243,NZ 9198 /VA 8084
4,2024-05-31,UA,1003,Denver International,Denver,DEN,Boeing 757-300 Passenger,753,05:00,08:37:00,157,AC 4046 /NZ 9057 /VA 8335


In [65]:
df_flights_plane['DEST_CITY_NAME'] = df_flights_plane['DEST_CITY_NAME'].str.replace(r',.*', '', regex=True)
df_flights_plane['ORIGIN_CITY_NAME'] = df_flights_plane['ORIGIN_CITY_NAME'].str.replace(r',.*', '', regex=True)

/var/folders/hx/g2lg_kq17xgb1zndw9jlbyn80000gn/T/ipykernel_31797/506844661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flights_plane['DEST_CITY_NAME'] = df_flights_plane['DEST_CITY_NAME'].str.replace(r',.*', '', regex=True)
/var/folders/hx/g2lg_kq17xgb1zndw9jlbyn80000gn/T/ipykernel_31797/506844661.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_flights_plane['ORIGIN_CITY_NAME'] = df_flights_plane['ORIGIN_CITY_NAME'].str.replace(r',.*', '', regex=True)


In [66]:

# Dropping the specified columns
df_future_flights = df_future_flights.drop(columns=[
    'flightnumber', 
    'airport_name', 
    'airport_IATA', 
    'aircraft_name', 
    'arrival_time', 
    'codeshare_info',
    'carrier'
])

# Renaming the remaining columns
df_future_flights = df_future_flights.rename(columns={
    'clean_date': 'FL_DATE',
    'airport_city_name': 'DEST_CITY_NAME',
    'aircraft_IATA': 'PLANE_MODEL',
    'departure_time': 'DEP_TIME',
    'elapsed_time': 'AIR_TIME'
})


In [67]:
columns_to_drop = [
    'ORIGIN_AIRPORT_ID', 
    'OP_UNIQUE_CARRIER', 
    'OP_CARRIER_FL_NUM', 
    'TAIL_NUM', 
    'YEAR', 
    'MONTH', 
    'DAY_OF_MONTH', 
    'DEST_AIRPORT_ID', 
    'DEP_DELAY',
    'ORIGIN_CITY_NAME'
]

df_flights_plane = df_flights_plane.drop(columns=columns_to_drop)

In [68]:
df_future_flights['FL_DATE'] = pd.to_datetime(df_future_flights['FL_DATE'])
df_future_flights['DAY_OF_WEEK'] = df_future_flights['FL_DATE'].dt.dayofweek
df_flights_plane['FL_DATE'] = pd.to_datetime(df_flights_plane['FL_DATE'])
df_flights_plane['DAY_OF_WEEK'] = df_flights_plane['FL_DATE'].dt.dayofweek

In [69]:
df_future_flights

,FL_DATE,DEST_CITY_NAME,PLANE_MODEL,DEP_TIME,AIR_TIME,DAY_OF_WEEK
0,2024-05-31,Houston,7M9,00:30,236,4
1,2024-05-31,Manila,77W,00:45,850,4
2,2024-05-31,Chicago,739,01:10,260,4
3,2024-05-31,Houston,7M9,05:00,243,4
4,2024-05-31,Denver,753,05:00,157,4
...,...,...,...,...,...,...
198,2024-05-31,Taipei,777,23:45,800,4
199,2024-05-31,Hong Kong,77W,23:50,850,4
200,2024-05-31,Seoul,777,23:55,745,4
201,2024-05-31,Houston,777,23:59,235,4


In [75]:
columns_order_future = ['FL_DATE', 'DAY_OF_WEEK', 'DEST_CITY_NAME', 'AIR_TIME', 'DEP_TIME', 'PLANE_MODEL']
df_future_flights = df_future_flights[columns_order_future]
columns_order_past = ['FL_DATE', 'DAY_OF_WEEK', 'DEST_CITY_NAME', 'AIR_TIME', 'DEP_TIME', 'PLANE_MODEL', 'DEP_DEL15']
df_flights_plane = df_flights_plane[columns_order_past]

# Change AIR_TIME type from int to float
df_future_flights['AIR_TIME'] = df_future_flights['AIR_TIME'].astype('float64')
df_flights_plane['DEP_DEL15'] = df_flights_plane['DEP_DEL15'].astype('int')

# Verify the column order and types
print(df_future_flights.info())
print(df_flights_plane.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   FL_DATE         203 non-null    datetime64[ns]
 1   DAY_OF_WEEK     203 non-null    int32         
 2   DEST_CITY_NAME  203 non-null    object        
 3   AIR_TIME        203 non-null    float64       
 4   DEP_TIME        203 non-null    object        
 5   PLANE_MODEL     203 non-null    object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 8.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 50239 entries, 0 to 50239
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   FL_DATE         50239 non-null  datetime64[ns]
 1   DAY_OF_WEEK     50239 non-null  int32         
 2   DEST_CITY_NAME  50239 non-null  object        
 3   AIR_TIME        50239 non-